# Data-Collecting

I will read the data from the XML & JSON files and process them accordingly. Then finally I will save them in final .csv files.


# Types of Data

*   Apple Health Data (Step Counts, Flights Climbed, Walking/Running Distance)
*   Period Tracker App (Dates of menstrual, follicular, luteal, and ovulation phases)

In [ ]:
# Data paths
apple_data_path = "/content/Raw_Data/export.xml"
clue_data_path = "/content/Raw_Data/measurements.json"
output_path = "/content/Output"

## Apple Health

This data holds my daily step count, flights climbed, walking/running distance information.

In [9]:
import os
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

# XML file path
xml_file_path = "/content/Raw_Data/export.xml"

# Date filter
start_date_filter = datetime.strptime("2021-05-07", "%Y-%m-%d")

# List to store health data records
all_records = []

# Load the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Extract data and normalize units
for record in root.findall("Record"):
    record_type = record.get("type", "Unknown").replace("HKQuantityTypeIdentifier", "")
    start_date = record.get("startDate", "Unknown")
    value = record.get("value", "Unknown")

    try:
        record_date = datetime.strptime(start_date.split(" ")[0], "%Y-%m-%d")
        if record_date >= start_date_filter and record_type in {
            "StepCount",
            "DistanceWalkingRunning",
            "FlightsClimbed"
        }:
            value = float(value)  # Keep the raw value in meters for DistanceWalkingRunning

            all_records.append({
                "date": record_date.strftime("%Y-%m-%d"),
                "type": record_type,
                "value": value
            })
    except ValueError:
        continue  # Skip invalid records

# Convert the data to a DataFrame
df = pd.DataFrame(all_records)

# Ensure only one record per date and type
df = df.groupby(["date", "type"], as_index=False).agg({"value": "sum"})

# Generate a complete list of dates and types
all_dates = pd.date_range(start=df["date"].min(), end=df["date"].max())
all_types = ["StepCount", "FlightsClimbed", "DistanceWalkingRunning"]

# Create a full grid of dates and types
full_grid = pd.DataFrame([(d.strftime("%Y-%m-%d"), t) for d in all_dates for t in all_types], columns=["date", "type"])

# Merge the full grid with the existing data
df = pd.merge(full_grid, df, on=["date", "type"], how="left").fillna(0)

# Output directory and file path
output_dir = "/content/Output"
output_file = os.path.join(output_dir, "apple_health_combined_data.csv")

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the data to a CSV file
df.to_csv(output_file, index=False, columns=["date", "type", "value"])
print(f"Final data with consistent units and complete days has been saved to {output_file}")

Final data with consistent units and complete days has been saved to /content/Output/apple_health_combined_data.csv


In [42]:
import pandas as pd

# Load the CSV file
output_file = "/content/Output/apple_health_combined_data.csv"
df = pd.read_csv(output_file)

# Group by date and type, summing up the values for each day
df_grouped = df.groupby(['date', 'type'])['value'].sum().reset_index()

# Pivot the table to make each 'type' a column, with dates as rows
df_pivot = df_grouped.pivot(index='date', columns='type', values='value').reset_index()

# Replace NaN with 0 for missing values
df_pivot.fillna(0, inplace=True)

# Save the pivoted DataFrame to a new Excel file
output_excel_file = "/content/Excels/apple_health_combined_data.xlsx"
df_pivot.to_excel(output_excel_file, index=False, engine='openpyxl')

print(f"Grouped data has been saved to {output_excel_file}.")

# Load the Excel file and print its contents
df_loaded = pd.read_excel(output_excel_file, engine='openpyxl')
print("Contents of the Excel file:")
print(df_loaded.to_string(index=False))

Grouped data has been saved to /content/Excels/apple_health_combined_data.xlsx.
Contents of the Excel file:
      date  DistanceWalkingRunning  FlightsClimbed  StepCount
2021-05-07                1.496920               2       2027
2021-05-08                0.016740               0         30
2021-05-09                0.082990               0        126
2021-05-10                0.689040               3        858
2021-05-11                3.856890               6       5083
2021-05-12                0.798250               9       1112
2021-05-13                0.797200               6       1158
2021-05-14                1.211370               5       1633
2021-05-15                0.191820               4        266
2021-05-16                1.157140               2       1605
2021-05-17                8.516260               5      11314
2021-05-18                3.297640               2       5250
2021-05-19                1.468000               4       2218
2021-05-20              

### Step Counts & Flights Climbed & Walking/Running Distance

In [11]:
import os
import pandas as pd
from datetime import datetime
import xml.etree.ElementTree as ET

# Path to the XML file
xml_file_path = "/content/Raw_Data/export.xml"

# Date filter
start_date_filter = datetime.strptime("2021-05-07", "%Y-%m-%d")

# Dictionary to categorize health data by type
data_by_type = {
    "HKQuantityTypeIdentifierStepCount": [],
    "HKQuantityTypeIdentifierDistanceWalkingRunning": [],
    "HKQuantityTypeIdentifierFlightsClimbed": []
}

# Load the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Extract data and categorize it by type
for record in root.findall("Record"):
    record_type = record.get("type", "Unknown")
    start_date = record.get("startDate", "Unknown")
    value = record.get("value", "Unknown")

    try:
        record_date = datetime.strptime(start_date.split(" ")[0], "%Y-%m-%d")
        if record_date >= start_date_filter and record_type in data_by_type:
            data_by_type[record_type].append({
                "date": record_date.strftime("%Y-%m-%d"),
                "value": float(value)  # Convert value to float for aggregation
            })
    except ValueError:
        continue  # Skip records with invalid dates

# Output directory
output_dir = "/content/Output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create a separate CSV file for each data type
for data_type, records in data_by_type.items():
    if records:  # If data exists for this type
        df = pd.DataFrame(records)

        # Group by date and aggregate values (sum if multiple entries exist for a single date)
        df = df.groupby("date", as_index=False).agg({"value": "sum"})

        # Save to CSV
        output_file = os.path.join(output_dir, f"{data_type.lower().replace('hkquantitytypeidentifier', '')}_data.csv")
        df.to_csv(output_file, index=False)
        print(f"{data_type} data has been saved to {output_file}")
    else:
        print(f"No data found for {data_type}")

HKQuantityTypeIdentifierStepCount data has been saved to /content/Output/stepcount_data.csv
HKQuantityTypeIdentifierDistanceWalkingRunning data has been saved to /content/Output/distancewalkingrunning_data.csv
HKQuantityTypeIdentifierFlightsClimbed data has been saved to /content/Output/flightsclimbed_data.csv


In [24]:
import pandas as pd

# File paths for the uploaded CSV files
flights_file = '/content/Output/flightsclimbed_data.csv'
steps_file = '/content/Output/stepcount_data.csv'
distance_file = '/content/Output/distancewalkingrunning_data.csv'

# Reading the CSV files into pandas DataFrames
distance_df = pd.read_csv(distance_file)
flights_df = pd.read_csv(flights_file)
steps_df = pd.read_csv(steps_file)

# Formatting and displaying the output
print("=== Step Count Data ===")
print(steps_df.to_string(index=False, header=['Date', 'Step Count']))  # Adjust headers
print("\n")

print("=== Flights Climbed Data ===")
print(flights_df.to_string(index=False, header=['Date', 'Flights Climbed']))  # Adjust headers
print("\n")

print("=== Distance Walking Running Data ===")
print(distance_df.to_string(index=False, header=['Date', 'Distance (km)']))  # Adjust headers

=== Step Count Data ===
      Date Step Count
2021-05-07     2027.0
2021-05-08       30.0
2021-05-09      126.0
2021-05-10      858.0
2021-05-11     5083.0
2021-05-12     1112.0
2021-05-13     1158.0
2021-05-14     1633.0
2021-05-15      266.0
2021-05-16     1605.0
2021-05-17    11314.0
2021-05-18     5250.0
2021-05-19     2218.0
2021-05-20     2772.0
2021-05-21     6280.0
2021-05-22       95.0
2021-05-23      165.0
2021-05-24     4706.0
2021-05-25     4603.0
2021-05-26     1326.0
2021-05-27     5159.0
2021-05-28     3318.0
2021-05-29      174.0
2021-05-30     5249.0
2021-05-31     6483.0
2021-06-01     5595.0
2021-06-02     2666.0
2021-06-03      113.0
2021-06-04      103.0
2021-06-05      278.0
2021-06-07      241.0
2021-06-08     9206.0
2021-06-09     2690.0
2021-06-10       51.0
2021-06-11     7535.0
2021-06-12     3879.0
2021-06-13      378.0
2021-06-14      218.0
2021-06-15      176.0
2021-06-16     2039.0
2021-06-17      197.0
2021-06-18     2380.0
2021-06-19     3631.0
2021-06-

In [26]:
import pandas as pd

# File paths for the uploaded CSV files
flights_file = '/content/Output/flightsclimbed_data.csv'
steps_file = '/content/Output/stepcount_data.csv'
distance_file = '/content/Output/distancewalkingrunning_data.csv'

# Reading the CSV files into pandas DataFrames
distance_df = pd.read_csv(distance_file)
flights_df = pd.read_csv(flights_file)
steps_df = pd.read_csv(steps_file)

# Renaming value columns for clarity
distance_df.rename(columns={"value": "Distance (km)"}, inplace=True)
flights_df.rename(columns={"value": "Flights Climbed"}, inplace=True)
steps_df.rename(columns={"value": "Step Count"}, inplace=True)

# Merging all datasets on the 'date' column
merged_df = steps_df.merge(flights_df, on="date", how="outer").merge(distance_df, on="date", how="outer")

# Fill any missing values with 0
merged_df.fillna(0, inplace=True)

# Sorting by date for a cleaner output
merged_df.sort_values(by="date", inplace=True)

# Displaying the merged dataset
print("=== Combined Data ===")
print(merged_df.to_string(index=False))

=== Combined Data ===
      date  Step Count  Flights Climbed  Distance (km)
2021-05-07      2027.0              2.0       1.496920
2021-05-08        30.0              0.0       0.016740
2021-05-09       126.0              0.0       0.082990
2021-05-10       858.0              3.0       0.689040
2021-05-11      5083.0              6.0       3.856890
2021-05-12      1112.0              9.0       0.798250
2021-05-13      1158.0              6.0       0.797200
2021-05-14      1633.0              5.0       1.211370
2021-05-15       266.0              4.0       0.191820
2021-05-16      1605.0              2.0       1.157140
2021-05-17     11314.0              5.0       8.516260
2021-05-18      5250.0              2.0       3.297640
2021-05-19      2218.0              4.0       1.468000
2021-05-20      2772.0              5.0       1.873270
2021-05-21      6280.0              5.0       4.135370
2021-05-22        95.0              1.0       0.072260
2021-05-23       165.0              3.0    

## Period Tracker App (Clue)

This data holds my dates of menstrual, follicular, luteal, and ovulation phases.

### Period Dates

Extracts and processes the data from the period tracker JSON file, filtering for period entries and saving it as a CSV.

In [15]:
import json
import csv
import os
from datetime import datetime, timedelta

# Path to the JSON file
json_file_path = "/content/Raw_Data/measurements.json"

# Path to the output CSV file
csv_file_path = os.path.join("/content/Output", "all_dates_with_period_flags.csv")

# Read the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Extract and sort period dates
period_dates = [datetime.strptime(item["date"], "%Y-%m-%d") for item in data if item["type"] == "period"]
period_dates = sorted(period_dates)

# Generate all dates from the first period date to the last period date
start_date = period_dates[0]
end_date = period_dates[-1]
all_dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Create a list with all dates and period flags
all_dates_with_flags = [
    {
        "date": current_date.strftime("%Y-%m-%d"),
        "is_period": 1 if current_date in period_dates else 0
    }
    for current_date in all_dates
]

# Write the data to a CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    # Write the header row
    writer.writerow(["date", "is_period"])
    # Write the data rows
    for item in all_dates_with_flags:
        writer.writerow([item["date"], item["is_period"]])

print(f"CSV file with all dates and period flags has been successfully saved to {csv_file_path}.")

CSV file with all dates and period flags has been successfully saved to /content/Output/all_dates_with_period_flags.csv.


In [27]:
import pandas as pd

# Path to the output CSV file
csv_file_path = "/content/Output/all_dates_with_period_flags.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the data in a tabular format
print("=== All Dates with Period Flags ===")
print(df.to_string(index=False))  # Display the DataFrame without the index

=== All Dates with Period Flags ===
      date  is_period
2021-05-07          1
2021-05-08          1
2021-05-09          1
2021-05-10          1
2021-05-11          0
2021-05-12          0
2021-05-13          0
2021-05-14          0
2021-05-15          0
2021-05-16          0
2021-05-17          0
2021-05-18          0
2021-05-19          0
2021-05-20          0
2021-05-21          0
2021-05-22          0
2021-05-23          0
2021-05-24          0
2021-05-25          0
2021-05-26          0
2021-05-27          0
2021-05-28          0
2021-05-29          0
2021-05-30          0
2021-05-31          0
2021-06-01          0
2021-06-02          0
2021-06-03          1
2021-06-04          1
2021-06-05          1
2021-06-06          1
2021-06-07          1
2021-06-08          0
2021-06-09          0
2021-06-10          0
2021-06-11          0
2021-06-12          0
2021-06-13          0
2021-06-14          0
2021-06-15          0
2021-06-16          0
2021-06-17          0
2021-06-18        

### Monthly Phases

Computes phase ranges (e.g., follicular, luteal, ovulation, menstruation) based on the period dates from the first script and saves the results.

In [17]:
import json
import csv
import os
from datetime import datetime, timedelta

# Path to the JSON file
json_file_path = "/content/Raw_Data/measurements.json"

# Path to the output CSV file
csv_file_path = os.path.join("/content/Output", "phase_date_ranges.csv")

# Read the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Sort the data by dates
data.sort(key=lambda x: datetime.strptime(x["date"], "%Y-%m-%d"))

# Calculate the phases
phases = []
for i in range(len(data) - 1):
    start_date = datetime.strptime(data[i]["date"], "%Y-%m-%d")
    end_date = datetime.strptime(data[i + 1]["date"], "%Y-%m-%d")
    duration = (end_date - start_date).days

    # Menstruation Phase
    menstruation_end = start_date + timedelta(days=5)  # Typically 5 days
    phases.append(["Menstruation", start_date.strftime("%Y-%m-%d"), menstruation_end.strftime("%Y-%m-%d")])

    # Follicular Phase
    follicular_start = menstruation_end + timedelta(days=1)
    ovulation_start = follicular_start + timedelta(days=9)  # Ovulation after 9 days
    phases.append(["Follicular", follicular_start.strftime("%Y-%m-%d"), ovulation_start.strftime("%Y-%m-%d")])

    # Ovulation Phase
    ovulation_end = ovulation_start + timedelta(days=1)  # Ovulation lasts 1 day
    phases.append(["Ovulation", ovulation_start.strftime("%Y-%m-%d"), ovulation_end.strftime("%Y-%m-%d")])

    # Luteal Phase
    luteal_start = ovulation_end + timedelta(days=1)
    luteal_end = end_date
    phases.append(["Luteal", luteal_start.strftime("%Y-%m-%d"), luteal_end.strftime("%Y-%m-%d")])

# Write the phase date ranges to a CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    # Write the header row
    writer.writerow(["Phase", "Start Date", "End Date"])
    # Write the data
    for phase in phases:
        writer.writerow(phase)

print(f"Phase date ranges have been successfully saved to {csv_file_path}.")

Phase date ranges have been successfully saved to /content/Output/phase_date_ranges.csv.


In [35]:
import pandas as pd

# Path to the CSV file
csv_file_path = "/content/Output/phase_date_ranges.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Separate the phases into individual DataFrames for side-by-side alignment
menstruation_df = df[df['Phase'] == 'Menstruation'][['Start Date', 'End Date']].reset_index(drop=True)
follicular_df = df[df['Phase'] == 'Follicular'][['Start Date', 'End Date']].reset_index(drop=True)
ovulation_df = df[df['Phase'] == 'Ovulation'][['Start Date', 'End Date']].reset_index(drop=True)
luteal_df = df[df['Phase'] == 'Luteal'][['Start Date', 'End Date']].reset_index(drop=True)

# Combine Start and End Dates into a single column for each phase
menstruation_df['Menstruation Start-End'] = menstruation_df['Start Date'] + " - " + menstruation_df['End Date']
follicular_df['Follicular Start-End'] = follicular_df['Start Date'] + " - " + follicular_df['End Date']
ovulation_df['Ovulation Start-End'] = ovulation_df['Start Date'] + " - " + ovulation_df['End Date']
luteal_df['Luteal Start-End'] = luteal_df['Start Date'] + " - " + luteal_df['End Date']

# Select only the merged columns
menstruation_df = menstruation_df[['Menstruation Start-End']]
follicular_df = follicular_df[['Follicular Start-End']]
ovulation_df = ovulation_df[['Ovulation Start-End']]
luteal_df = luteal_df[['Luteal Start-End']]

# Combine all DataFrames side by side
combined_df = pd.concat([menstruation_df, follicular_df, ovulation_df, luteal_df], axis=1)

# Display the DataFrame
print("=== Phase Date Ranges ===")
print(combined_df.to_string(index=False))

=== Phase Date Ranges ===
 Menstruation Start-End    Follicular Start-End     Ovulation Start-End        Luteal Start-End
2021-05-07 - 2021-05-12 2021-05-13 - 2021-05-22 2021-05-22 - 2021-05-23 2021-05-24 - 2021-05-08
2021-05-08 - 2021-05-13 2021-05-14 - 2021-05-23 2021-05-23 - 2021-05-24 2021-05-25 - 2021-05-09
2021-05-09 - 2021-05-14 2021-05-15 - 2021-05-24 2021-05-24 - 2021-05-25 2021-05-26 - 2021-05-10
2021-05-10 - 2021-05-15 2021-05-16 - 2021-05-25 2021-05-25 - 2021-05-26 2021-05-27 - 2021-06-03
2021-06-03 - 2021-06-08 2021-06-09 - 2021-06-18 2021-06-18 - 2021-06-19 2021-06-20 - 2021-06-04
2021-06-04 - 2021-06-09 2021-06-10 - 2021-06-19 2021-06-19 - 2021-06-20 2021-06-21 - 2021-06-05
2021-06-05 - 2021-06-10 2021-06-11 - 2021-06-20 2021-06-20 - 2021-06-21 2021-06-22 - 2021-06-06
2021-06-06 - 2021-06-11 2021-06-12 - 2021-06-21 2021-06-21 - 2021-06-22 2021-06-23 - 2021-06-07
2021-06-07 - 2021-06-12 2021-06-13 - 2021-06-22 2021-06-22 - 2021-06-23 2021-06-24 - 2021-06-29
2021-06-29 - 2